# Wrythm – Data Extraction and Dataset

## Notebook Objective
This notebook focuses primarily on:

1. Extracting audio.
2. Extracting relevant musical features (MFCCs, BPM, rhythmic, melodic patterns, and key).
3. Building a consolidated dataset in JSON format for future use in genre classification.
4. Initial data exploration (EDA – exploratory data analysis).

---

## Pipeline 

### 1. Audio Collection
- 
- 

### 2. Feature Extraction
For each collected audio, extract:

- **MFCCs**: mean and variance to capture timbre.
- **BPM**: beats per minute (song tempo).
- **Rhythmic Cell**: onset histogram divided into bins.
- **Melodic Cell**: dominant notes per segment.
- **Key**: estimated musical key.
- **Duration**: audio duration in seconds.

Features will be stored together with metadata in JSON format.

### 3. Dataset Construction
- Consolidate JSON files into a single folder (`data/metadata/`).
- Ensure each record contains:
  - Track identifier (Spotify ID)
  - Track name and artists
  - Audio file path
  - Extracted features
  - Genre(s)

### 4. Initial Exploratory Analysis (EDA)
- Count of tracks per genre.
- Distribution of duration, BPM, and key.
- Basic statistics of MFCCs and rhythmic patterns.
- Check for consistency and possible missing data.


# Development


## 1 - Initial Setup: Imports and Preliminaries

In [ ]:
# Selenium imports and helpers
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    TimeoutException,
    NoSuchElementException,
    ElementClickInterceptedException,
    StaleElementReferenceException,
    WebDriverException,
 )
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time


## 2 - Data Collection & Feature Extraction


In [ ]:
# Start Chrome (adjust options as needed)
options = Options()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--window-size=1280,1600')
# options.add_argument('--headless=new')  # uncomment to run headless
driver = webdriver.Chrome(options=options)

url = "https://prosearch.tribeofnoise.com/search/index?_ga=2.22448313.302227877.1760491446-1089789772.1760491446&_gl=1*1baaiwo*_ga*MTA4OTc4OTc3Mi4xNzYwNDkxNDQ2*_ga_JQ9GCR3CSF*czE3NjA0OTE0NDUkbzEkZzAkdDE3NjA0OTE0NDUkajYwJGwwJGgw"
driver.get(url)
wait = WebDriverWait(driver, 20)  # longer default wait

# helper: retorna o número de itens já carregados no container (ajuste o seletor conforme o DOM)
def count_results():
    try:
        # ajuste o seletor abaixo para o seletor real dos resultados na página
        elems = driver.find_elements(By.CSS_SELECTOR, '#search-results .search-result-item, .search-result-item')
        return len(elems)
    except Exception:
        return 0

def find_load_more_button_locator():
    """Return a tuple (by, selector) for the 'Load more' button if found, else None."""
    try_locators = [
        (By.XPATH, "//button[@data-load-more]"),
        (By.XPATH, "//button[contains(., 'Load more')]"),
        (By.XPATH, "//*[@id='search-results']/p/button"),
        (By.XPATH, "//p/button[contains(@class,'c-button')]")
    ]
    for locator in try_locators:
        try:
            elems = driver.find_elements(*locator)
            if elems:
                return locator
        except Exception:
            continue
    return None

def get_element(locator):
    by_, sel = locator
    return driver.find_element(by_, sel)

# main loop: rolar até o botão, clicar e esperar novos itens aparecerem
max_rounds = 120  # mais rodadas para garantir
min_rounds_before_exit = max_rounds // 2  # não sair antes de metade
round_idx = 0
loaded = count_results()
print('Resultados iniciais:', loaded)
stalled_rounds = 0
no_button_rounds = 0
same_loaded_rounds = 0
no_scroll_growth_rounds = 0
last_scroll_height = driver.execute_script('return document.body.scrollHeight')

def robust_scroll(max_tries=6, pause=0.25):
    last_height = driver.execute_script('return document.body.scrollHeight')
    tries = 0
    while tries < max_tries:
        driver.execute_script('window.scrollBy(0, Math.max(document.documentElement.clientHeight, 800));')
        time.sleep(pause)
        new_height = driver.execute_script('return document.body.scrollHeight')
        if new_height == last_height:
            tries += 1
        else:
            last_height = new_height
            tries = 0
    return last_height

while round_idx < max_rounds:
    round_idx += 1
    print(f'Rodada {round_idx}: rolando até o fim da página...')
    # tentar rolar o container de resultados se existir (mais eficiente em páginas SPA)
    try:
        has_container = driver.execute_script("return !!document.querySelector('#search-results');")
    except Exception:
        has_container = False
    if has_container:
        try:
            driver.execute_script("var c=document.querySelector('#search-results'); c.scrollTop = c.scrollHeight;")
            time.sleep(0.2)
        except Exception as e:
            print('Erro ao rolar container #search-results:', e)
            robust_scroll(max_tries=4, pause=0.25)
    else:
        robust_scroll(max_tries=6, pause=0.25)

    # medir crescimento de scrollHeight entre rodadas
    try:
        current_scroll_height = driver.execute_script('return document.body.scrollHeight')
        if current_scroll_height <= last_scroll_height:
            no_scroll_growth_rounds += 1
        else:
            no_scroll_growth_rounds = 0
            last_scroll_height = current_scroll_height
    except Exception:
        pass

    # localizar o botão 'Load more'
    btn_locator = find_load_more_button_locator()
    if btn_locator is None:
        no_button_rounds += 1
        print('Botão não encontrado nesta rodada (', no_button_rounds, '). Tentando novamente...')
        # Se não há botão, talvez chegamos ao fim; tente detectar estagnação por várias rodadas
        time.sleep(0.8)
        # Recontar para checar progresso automático
        new_count = count_results()
        if new_count == loaded:
            same_loaded_rounds += 1
        else:
            loaded = new_count
            same_loaded_rounds = 0
        # só sair se já passamos do mínimo de rodadas e houver múltiplos sinais de estagnação
        if (round_idx >= min_rounds_before_exit and
            no_button_rounds >= 6 and
            same_loaded_rounds >= 6 and
            no_scroll_growth_rounds >= 6):
            print('Vários sinais de estagnação sem botão; encerrando loop.')
            break
        else:
            continue
    else:
        # reset do contador de "sem botão" quando o botão aparece novamente
        no_button_rounds = 0

    # tentar clicar no botão encontrado
    try:
        btn = wait.until(EC.presence_of_element_located(btn_locator))
        driver.execute_script('arguments[0].scrollIntoView({block: "center"});', btn)
        # Espera o botão ficar clicável
        wait.until(EC.element_to_be_clickable(btn_locator))
        # Tentativa 1: click direto
        try:
            btn.click()
        except (ElementClickInterceptedException, StaleElementReferenceException):
            # Tentativa 2: ActionChains move + click
            try:
                ActionChains(driver).move_to_element(btn).pause(0.15).click(btn).perform()
            except Exception:
                # Tentativa 3: JS click
                driver.execute_script('arguments[0].click();', btn)
        print('Clique enviado; aguardando novos itens...')
    except (WebDriverException, TimeoutException) as e:
        print('Falha ao acionar clique (Action/JS fallback):', e)
        try:
            btn = get_element(btn_locator)
            driver.execute_script('arguments[0].click();', btn)
        except Exception as e2:
            print('JS click falhou:', e2)

    # esperar por novos items (comparar contagem)
    wait_start = time.time()
    timeout_s = 8  # mais tempo para redes lentas
    round_loaded_before = loaded
    while time.time() - wait_start < timeout_s:
        time.sleep(0.5)
        cur = count_results()
        if cur > loaded:
            print('Novos itens carregados:', cur - loaded)
            loaded = cur
            stalled_rounds = 0
            same_loaded_rounds = 0
            no_scroll_growth_rounds = 0
            break
    else:
        print('Nenhum novo item detectado após o clique nesta rodada.')
        stalled_rounds += 1
        if loaded == round_loaded_before:
            same_loaded_rounds += 1
        # só sair por estagnação de clique se já passamos do mínimo de rodadas
        if (round_idx >= min_rounds_before_exit and (stalled_rounds >= 6 or same_loaded_rounds >= 8)):
            print('Sem progresso após cliques por várias rodadas; terminando.')
            break

    # pequena espera antes da próxima rodada
    time.sleep(0.25)

print('Loop finalizado. Total de itens carregados:', loaded)
# salvar screenshot final
try:
    driver.save_screenshot('final_results.png')
    print('Screenshot salvo em final_results.png')
except Exception as e:
    print('Falha ao salvar screenshot final:', e)

# note: call driver.quit() when done if you don't need the browser open


Resultados iniciais: 0
Rodada 1: rolando até o fim da página...
Clique enviado; aguardando novos itens...
Clique enviado; aguardando novos itens...
Nenhum novo item detectado após o clique nesta rodada.
Nenhum novo item detectado após o clique nesta rodada.
Rodada 2: rolando até o fim da página...
Rodada 2: rolando até o fim da página...
Clique enviado; aguardando novos itens...
Clique enviado; aguardando novos itens...
Nenhum novo item detectado após o clique nesta rodada.
Nenhum novo item detectado após o clique nesta rodada.
Rodada 3: rolando até o fim da página...
Rodada 3: rolando até o fim da página...
Clique enviado; aguardando novos itens...
Clique enviado; aguardando novos itens...
Nenhum novo item detectado após o clique nesta rodada.
Nenhum novo item detectado após o clique nesta rodada.
Rodada 4: rolando até o fim da página...
Rodada 4: rolando até o fim da página...
Clique enviado; aguardando novos itens...
Clique enviado; aguardando novos itens...
Nenhum novo item detecta

In [ ]:
# Optional cleanup: safely close the browser
try:
    driver.quit()
    print('Driver encerrado com sucesso.')
except Exception as e:
    print('Erro ao encerrar driver:', e)